# Oscilloskope utility – using Ethernet with GPS timemarks 

In [4]:
import matplotlib.pyplot as plt
import sys
import os
import time
import h5py
import numpy as np
import glob
import vxi11
import serial
import struct
import math

# pyUblox libraries
import ublox
import util
import datetime

gpsport = '/dev/tty.usbmodem1411'
gpsbaudrate = 115200

# Step 0:
# Connect oscilloscope via direct Ethernet link
# Step 1:
# Run this cell to verify the connection.

# This will need a rewrite
class TmcDriver:

    def __init__(self, device):
        print("Initializing connection to: " + device)
        self.device = device
        self.instr = vxi11.Instrument(device)
 
    def write(self, command):
        self.instr.write(command);

    def read(self, length = 500):
        return self.instr.read(length)

    def read_raw(self, length = 500):
        return self.instr.read_raw(length)
 
    def getName(self):
        self.write("*IDN?")
        return self.read(300)
    
    def ask(self, command):
        return self.instr.ask(command)
 
    def sendReset(self):
        self.write("*RST")  # Be carefull, this real resets an oscilloscope
            
# For Ethernet
#osc = TmcDriver("TCPIP::147.231.24.72::INSTR")
osc = TmcDriver("TCPIP::10.1.1.254::INSTR")
print(osc.ask("*IDN?"))

Initializing connection to: TCPIP::10.1.1.254::INSTR
RIGOL TECHNOLOGIES,DS2102A,DS2D174501960,00.03.04.SP2



## Read repeatedly records from oscilloscope

In [12]:
filename = 0

if (filename == 1):
    for f in glob.iglob("./data/*.h5"): # delete all .h5 files 
        print 'Deleting', f
        os.remove(f)
else:
    print 'Not removing old files, as filename {0} is not 1.'.format(filename)

# Initialize GPS grabber
print 'Configuring GPS'
dev = ublox.UBlox(gpsport, baudrate=gpsbaudrate, timeout=1)
dev.set_logfile(None, append=True)
dev.set_binary()
dev.configure_poll_port()
dev.configure_poll(ublox.CLASS_CFG, ublox.MSG_CFG_USB)
dev.configure_poll(ublox.CLASS_MON, ublox.MSG_MON_HW)
dev.configure_port(port=ublox.PORT_SERIAL1, inMask=1, outMask=0)
dev.configure_port(port=ublox.PORT_USB, inMask=1, outMask=1)
dev.configure_port(port=ublox.PORT_SERIAL2, inMask=1, outMask=0)
dev.configure_poll_port()
dev.configure_poll_port(ublox.PORT_SERIAL1)
dev.configure_poll_port(ublox.PORT_SERIAL2)
dev.configure_poll_port(ublox.PORT_USB)
dev.configure_solution_rate(rate_ms=100)

dev.set_preferred_dynamic_model(ublox.DYNAMIC_MODEL_STATIONARY)
dev.set_preferred_usePPP(None)

dev.configure_message_rate(ublox.CLASS_NAV, ublox.MSG_NAV_POSLLH, 0)
dev.configure_message_rate(ublox.CLASS_NAV, ublox.MSG_NAV_POSECEF, 0)
dev.configure_message_rate(ublox.CLASS_RXM, ublox.MSG_RXM_RAW, 0)
dev.configure_message_rate(ublox.CLASS_RXM, ublox.MSG_RXM_SFRB, 0)

# Poll UTC time
dev.configure_message_rate(ublox.CLASS_NAV, ublox.MSG_NAV_TIMEUTC, 0)
# Poll TM2 messages
dev.configure_message_rate(ublox.CLASS_TIM, ublox.MSG_TIM_TM2,1)

while True:
    # Start the run
    #print 'Exit recording mode if set'
    osc.write(':FUNC:WRM OFF')
    
    print 'Waiting for TM2 message'
    zero_count = None
    
    run_start_time = time.time()
        
    dev.close()
    dev = ublox.UBlox(gpsport, baudrate=gpsbaudrate, timeout=1)
    dev.set_logfile(None, append=True)
    
    # Wait for one spill
    while True:
        msg = dev.receive_message()
        if (msg is None) and (zero_count is None):
            print '  Waiting for signal.'
            dev.close()
            dev = ublox.UBlox(gpsport, baudrate=gpsbaudrate, timeout=1)
            dev.set_logfile(None, append=True)
        elif (msg is None):
            break
        else:
            try:
                #gpslog = open(logfile_base + '.gps', 'a+')       
                #gpslog.write('# '+ str(msg) + '\n')
                #gpslog.close()
                True
            except ublox.UBloxError as e:
                print(e)
            if msg.name() == 'TIM_TM2':
                #print('Got TM2 message')
                try:
                    msg.unpack()
                    if (zero_count is None):
                        zero_count = msg.count - 1
                        last_count = msg.count
                        first_towMsR = msg.towMsR
                        count_base = 0;
                        #print('  First counter value: ' + str(msg.count))
                    if (msg.count < zero_count+1):
                        count_base += 65535;
                    timestring = datetime.datetime.utcfromtimestamp(util.gpsTimeToTime(msg.wnR, 1.0e-3*msg.towMsR))
                    #txt = "Counter = %5d Zero = %5d Diff = %5d %s" % (msg.count, zero_count, msg.count-last_count, timestring)
                    #print txt
                    sys.stdout.write('%d ' % msg.count)
                    last_count = msg.count
                except ublox.UBloxError as e:
                    print(e)

    print "=> TM2 Counter initialized: %5d %s\n" % (last_count, timestring)
    zero_count = last_count
    
    #zero_wnR = msg.wnR
    #zero_towMsR = msg.towMsR
    #zero_towSubMsR = msg.towSubMsR
    
    # Stop the oscilloscope
    #osc.write(':STOP') # Stop button
    #time.sleep(0.2)
    
    # Reopen GPS port
    dev.close()
    dev = ublox.UBlox(gpsport, baudrate=gpsbaudrate, timeout=0)
    dev.set_logfile(None, append=True)
    
    osc.write(':FUNC:WREC:OPER REC') # start recording
    subrun_start_time = time.time()
    print '  Capturing data from oscilloscope...'
    time.sleep(0.2)
    
    # Setup log file base name
    logfile_base = './data/data'+'_'+str(int(round(subrun_start_time,0)))+'_{:02.0f}'.format(filename)
    
    while True:
        # Read GPS messages, if any
        while True:
            msg = dev.receive_message()
            if (msg is None):
                break
            else:
                try:
                    # Open GPS log file
                    gpslog = open(logfile_base + '.gps', 'a+')       
                    gpslog.write('%d' % zero_count + ' ' + str(msg) + '\n')
                    gpslog.close()
                except ublox.UBloxError as e:
                    print(e)
                if msg.name() == 'TIM_TM2':
                    #print('Got TM2 message')
                    try:
                        msg.unpack()
                        if (msg.count < zero_count+1):
                            print "TM2 counter overflow"
                            count_base = 65535
                        else:
                            count_base = 0
                        timestring = datetime.datetime.utcfromtimestamp(util.gpsTimeToTime(msg.wnR, 1.0e-3*msg.towMsR))
                        #print 'Counter = %5d Absolute = %5d Zero = %5d Diff = %5d %s" % (msg.count, msg.count-zero_count, zero_count, msg.count-last_count, timestring)
                        sys.stdout.write('%d:%d ' % (msg.count-zero_count, msg.count-last_count))
                        sys.stdout.flush()
                        gpslog = open(logfile_base + '.tm2', 'a+')       
                        gpslog.write('%d %d %d %d %d %d "%s"\n' % (zero_count, msg.count, count_base + msg.count-zero_count, msg.wnR, msg.towMsR, msg.towSubMsR, timestring))
                        gpslog.close()
                        last_count = msg.count
                    except ublox.UBloxError as e:
                        print(e)
                        
        osc.write(':FUNC:WREC:OPER?') # finished recording?
        reply = osc.read()
        if reply == 'STOP':
            subrun_time = round(time.time() - subrun_start_time, 2)
            print('  Subrun finished after %.2f seconds.' % subrun_time)
            break
        time.sleep(0.01)

    # Data readout
    osc.write(':WAV:SOUR CHAN1')
    osc.write(':WAV:MODE NORM')
    osc.write(':WAV:FORM BYTE')
    osc.write(':WAV:POIN 1400')

    osc.write(':WAV:XINC?')
    xinc = float(osc.read(100))
    print 'XINC:', xinc,
    osc.write(':WAV:YINC?')
    yinc = float(osc.read(100))
    print 'YINC:', yinc,
    osc.write(':TRIGger:EDGe:LEVel?')
    trig = float(osc.read(100))
    print 'TRIG:', trig,
    osc.write(':WAVeform:YORigin?')
    yorig = float(osc.read(100))
    print 'YORIGIN:', yorig,
    osc.write(':WAVeform:XORigin?')
    xorig = float(osc.read(100))
    print 'XORIGIN:', xorig,
    
    osc.write(':FUNC:WREP:FEND?') # get number of last frame
    frames = int(osc.read(100))
    print 'FRAMES:', frames, 'SUBRUN', filename
    
    with h5py.File(logfile_base + '.h5', 'w') as hf:       
        hf.create_dataset('FRAMES', data=(frames)) # write number of frames
        hf.create_dataset('XINC', data=(xinc)) # write axis parameters
        hf.create_dataset('YINC', data=(yinc))
        hf.create_dataset('TRIG', data=(trig))
        hf.create_dataset('YORIGIN', data=(yorig))
        hf.create_dataset('XORIGIN', data=(xorig))
        hf.create_dataset('CAPTURING', data=(subrun_time))
        osc.write(':FUNC:WREP:FCUR 1') # skip to n-th frame
        time.sleep(0.5)
        for n in range(1,frames+1):
            osc.write(':FUNC:WREP:FCUR ' + str(n)) # skip to n-th frame
            time.sleep(0.001)

            osc.write(':WAV:DATA?') # read data
            #time.sleep(0.4)
            wave1 = bytearray(osc.read_raw(500))
            wave2 = bytearray(osc.read_raw(500))
            wave3 = bytearray(osc.read_raw(500))
            #wave4 = bytearray(osc.read(500))
            #wave = np.concatenate((wave1[11:],wave2[:(500-489)],wave3[:(700-489)]))
            wave = np.concatenate((wave1[11:],wave2,wave3[:-1]))
            hf.create_dataset(str(n), data=wave)

    # Increase filename prefix
    filename = filename + 1
    

Not removing old files, as filename 0 is not 1.
Configuring GPS
try set binary at 115200
Waiting for TM2 message
  Waiting for signal.
8566 8567 8567 8571 8577 8588 8601 8613 8625 8637 8646 => TM2 Counter initialized:  8646 2018-01-08 05:11:08.165000

  Capturing data from oscilloscope...
1:1 11:10 25:14 39:14 53:14 67:14 81:14 96:15 111:15 126:15   Subrun finished after 1.24 seconds.
XINC: 2e-07 YINC: 8.000001e-05 TRIG: 0.00068 YORIGIN: -68.0 XORIGIN: -4.479999e-05 FRAMES: 127 SUBRUN 0
Waiting for TM2 message
8783 8795 8808 8820 8832 8843 8855 8863 => TM2 Counter initialized:  8863 2018-01-08 05:11:18.068000

  Capturing data from oscilloscope...
13:13 26:13 38:12 50:12 61:11 73:12 81:8 91:10 102:11 113:11 115:2 116:1 118:2   Subrun finished after 5.05 seconds.
XINC: 2e-07 YINC: 8.000001e-05 TRIG: 0.00068 YORIGIN: -68.0 XORIGIN: -4.479999e-05 FRAMES: 127 SUBRUN 1
Waiting for TM2 message
  Waiting for signal.
  Waiting for signal.
8991 9004 9017 9030 9043 9057 9070 9083 9096 9109 9120 

KeyboardInterrupt: 

In [16]:
# Attempt to make a clickable interface
%gui asyncio
from ipywidgets import widgets
from IPython.display import display
import sys

def button_clicked(b):
    print 'Trigger bigger' + str(b)

start_button = widgets.Button(description='Start capture')
stop_button = widgets.Button(description='Stop capture')
hbox = widgets.HBox([start_button, stop_button])

start_button.on_click(button_clicked)
stop_button.on_click(button_clicked)

display(hbox)

Go = True

HBox(children=(Button(description=u'Start capture', style=ButtonStyle()), Button(description=u'Stop capture', style=ButtonStyle())))

In [1]:
#osc.write(':STOP')
#osc.write('*INF')
#osc.read()

In [17]:
import ublox # pyUblox librarie
import util
import datetime
import sys


gpsport = '/dev/ttyACM0'
gpsbaudrate = 9600

# Reopen GPS port
dev = ublox.UBlox(gpsport, baudrate=gpsbaudrate, timeout=0)

zero_count = 0
last_count = 0

# Read GPS messages, if any
while True:
    msg = dev.receive_message()
    if (msg is None):
        pass#break
    else:
        if msg.name() == 'TIM_TM2':
            #print('Got TM2 message')
            try:
                msg.unpack()
                if (msg.count < zero_count+1):
                    print "TM2 counter overflow"
                    count_base = 65535
                else:
                    count_base = 0
                timestring = '$HIT,'
                timestring += str(msg.count)
                timestring += ','
                timestring += str(datetime.datetime.utcfromtimestamp(util.gpsTimeToTime(msg.wnR, 1.0e-3*msg.towMsR)))
                #print 'Counter = %5d Absolute = %5d Zero = %5d Diff = %5d %s" % (msg.count, msg.count-zero_count, zero_count, msg.count-last_count, timestring)
                print timestring
                #sys.stdout.write('%d:%d ' % (msg.count-zero_count, msg.count-last_count))
                #sys.stdout.flush()
                last_count = msg.count
            except ublox.UBloxError as e:
                print(e)
            break;

dev.close()


$HIT,101,2019-06-01 10:37:16.027000
